## Github setup

In [1]:
from google.colab import drive, userdata
import os

# 1. Mount Drive
drive.mount('/content/drive')

# 2. Setup Paths (Change to your actual repo name)
REPO_PATH = "/content/drive/MyDrive/ML/DL_With_Pytorch"
%cd {REPO_PATH}

# 3. Secure Auth
token = userdata.get('GH_TOKEN')
username = "barada02"
repo = "DL_With_Pytorch"
!git remote set-url origin https://{token}@github.com/{username}/{repo}.git

# 4. Identity
!git config --global user.email "Chandanbarada2@gmail.com"
!git config --global user.name "Kumar"

!git pull origin main
print("✅ Environment Ready!")

Mounted at /content/drive
/content/drive/MyDrive/ML/DL_With_Pytorch
From https://github.com/barada02/DL_With_Pytorch
 * branch            main       -> FETCH_HEAD
Already up to date.
✅ Environment Ready!


## Commit and Push

In [2]:
# 2. Push notebook changes to GitHub
# IMPORTANT: Press Ctrl+S (Save) before running this!
!git add .
!git commit -m "autograd operation diagram "
!git push origin main

[main 6adb7fb] autograd operation diagram
 1 file changed, 1 insertion(+), 1578 deletions(-)
 rewrite 02_Workflow.ipynb (99%)
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 5.56 KiB | 407.00 KiB/s, done.
Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/barada02/DL_With_Pytorch.git
   eaac25e..6adb7fb  main -> main


# Workflow

# PyTorch Deep Learning Workflow

This notebook demonstrates the **complete end-to-end workflow** for training a neural network in PyTorch.

## 📋 Workflow Overview:
1. **Setup & Import** - Import necessary libraries
2. **Data Preparation** - Load and prepare datasets
3. **Model Definition** - Create neural network architecture
4. **Training Configuration** - Set hyperparameters, loss function, optimizer
5. **Training Loop** - Train the model
6. **Evaluation** - Test model performance
7. **Model Persistence** - Save and load trained models

This workflow is fundamental to all deep learning projects and can be adapted for various tasks.

## Step 1: Import Required Libraries

**Purpose**: Import PyTorch and related libraries needed for the deep learning workflow.

**Key Imports**:
- `torch` - Core PyTorch library
- `torch.nn` - Neural network modules and building blocks
- `torch.utils.data.DataLoader` - Efficient data loading with batching
- `torchvision.datasets` - Pre-built datasets (FashionMNIST in this case)
- `torchvision.transforms` - Data preprocessing and augmentation

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
print(torch.__version__)

2.9.0+cpu


In [ ]:

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

100%|██████████| 26.4M/26.4M [00:01<00:00, 16.2MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 282kB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 5.03MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 22.5MB/s]


## Step 2: Load and Prepare Data

**Purpose**: Download and prepare the FashionMNIST dataset for training and testing.

**What is FashionMNIST?**
- 70,000 grayscale images of clothing items (28x28 pixels)
- 10 classes: T-shirt, Trouser, Pullover, Dress, Coat, Sandal, Shirt, Sneaker, Bag, Ankle boot
- Split into 60,000 training and 10,000 test images

**Key Parameters**:
- `root="data"` - Directory to store the dataset
- `train=True/False` - Specify training or test set
- `download=True` - Automatically download if not present
- `transform=ToTensor()` - Convert PIL images to PyTorch tensors (0-1 range)

In [ ]:

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)


## Step 3: Create DataLoaders

**Purpose**: Wrap datasets in DataLoader for efficient batch processing during training.

**Why DataLoader?**
- Automatically batches data into manageable chunks
- Shuffles data between epochs (for training)
- Enables parallel data loading with multiple workers
- Memory efficient - loads only the current batch

**Batch Size = 64**: Each iteration processes 64 images at once, balancing memory usage and training speed.

In [ ]:


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


## Step 4: Define the Neural Network

**Purpose**: Create the model architecture that will learn to classify images.

**Architecture Breakdown**:
1. **Input Layer**: 28×28 = 784 pixels (flattened)
2. **Hidden Layer 1**: 784 → 512 neurons with ReLU activation
3. **Hidden Layer 2**: 512 → 512 neurons with ReLU activation
4. **Output Layer**: 512 → 10 neurons (one per class)

**Key Components**:
- `nn.Flatten()` - Converts 2D image (28×28) to 1D vector (784)
- `nn.Linear()` - Fully connected layers (neurons)
- `nn.ReLU()` - Activation function (introduces non-linearity)
- `nn.Sequential()` - Chains layers together

**Forward Pass**: Defines how data flows through the network from input to output.

In [ ]:

model = NeuralNetwork()
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

## Step 5: Instantiate the Model

**Purpose**: Create an instance of the neural network.

This creates an untrained model with randomly initialized weights. The model is now ready to be trained.

## Hyperparameters

Hyperparameters are adjustable parameters that let you control the model optimization process. Different hyperparameter values can impact model training and convergence rates [read more](https://docs.pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html)

We define the following hyperparameters for training:
* Number of Epochs - the number of times to iterate over the dataset

* Batch Size - the number of data samples propagated through the network before the parameters are updated

* Learning Rate - how much to update models parameters at each batch/epoch. Smaller values yield slow learning speed, while large values may result in unpredictable behavior during training.

In [ ]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## Step 6: Set Hyperparameters

**Purpose**: Configure the training process parameters.

**Hyperparameter Definitions**:
- **Learning Rate (1e-3 = 0.001)**: Controls how much to adjust weights after each batch
  - Too high → unstable training
  - Too low → slow learning
  
- **Batch Size (64)**: Number of samples processed before updating weights
  - Larger → more stable gradients, more memory
  - Smaller → faster updates, noisier gradients
  
- **Epochs (5)**: Number of times to iterate through the entire dataset
  - More epochs → better learning (but risk overfitting)

These are the "knobs" you tune to improve model performance!

## Optimization Loop
Once we set our hyperparameters, we can then train and optimize our model with an optimization loop. Each iteration of the optimization loop is called an epoch.

**Each epoch consists of two main parts:**
**The Train Loop** - iterate over the training dataset and try to converge to optimal parameters.

**The Validation/Test Loop** - iterate over the test dataset to check if model performance is improving.

Let’s briefly familiarize ourselves with some of the concepts used in the training loop. Jump ahead to see the Full Implementation of the optimization loop.

## Loss Function
When presented with some training data, our untrained network is likely not to give the correct answer. Loss function measures the degree of dissimilarity of obtained result to the target value, and it is the loss function that we want to minimize during training. To calculate the loss we make a prediction using the inputs of our given data sample and compare it against the true data label value.

Common loss functions include nn.MSELoss (Mean Square Error) for regression tasks, and nn.NLLLoss (Negative Log Likelihood) for classification. nn.CrossEntropyLoss combines `nn.LogSoftmax` and `nn.NLLLoss`.

We pass our model’s output logits to `nn.CrossEntropyLoss`, which will normalize the logits and compute the prediction error.

In [ ]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

## Step 7: Initialize Loss Function

**Purpose**: Measure how far the model's predictions are from actual labels.

**CrossEntropyLoss**:
- Best for multi-class classification (choosing 1 class from many)
- Combines LogSoftmax + NLLLoss
- Lower loss = better predictions

**How it works**:
- Compares predicted probabilities with true labels
- Penalizes confident wrong predictions more heavily
- Goal: Minimize this loss during training

Optimizer
Optimization is the process of adjusting model parameters to reduce model error in each training step. Optimization algorithms define how this process is performed (in this example we use Stochastic Gradient Descent). All optimization logic is encapsulated in the optimizer object. Here, we use the SGD optimizer; additionally, there are many different optimizers available in PyTorch such as ADAM and RMSProp, that work better for different kinds of models and data.

We initialize the optimizer by registering the model’s parameters that need to be trained, and passing in the learning rate hyperparameter.

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## Step 8: Initialize Optimizer

**Purpose**: Algorithm that updates model weights to minimize loss.

**SGD (Stochastic Gradient Descent)**:
- Classic optimization algorithm
- Updates weights based on gradients (derivatives of loss)
- "Descends" toward minimum loss

**Alternative Optimizers**:
- `Adam` - Adaptive learning rate, often faster convergence
- `RMSProp` - Good for recurrent networks
- `AdaGrad` - Adapts learning rate per parameter

The optimizer needs:
1. Model parameters to update
2. Learning rate to control update size

Inside the training loop, optimization happens in three steps:
Call optimizer.zero_grad() to reset the gradients of model parameters. Gradients by default add up; to prevent double-counting, we explicitly zero them at each iteration.

Backpropagate the prediction loss with a call to loss.backward(). PyTorch deposits the gradients of the loss w.r.t. each parameter.

Once we have our gradients, we call optimizer.step() to adjust the parameters by the gradients collected in the backward pass.

### The Three Steps of Optimization (per batch):

**Visual Flow**:
```
Data Batch → Forward Pass → Calculate Loss → Backward Pass → Update Weights → Repeat
```

1. **`optimizer.zero_grad()`**
   - Clears old gradients from previous batch
   - Prevents gradient accumulation

2. **`loss.backward()`**
   - Backpropagation: Calculates gradients for all parameters
   - Determines how to adjust each weight

3. **`optimizer.step()`**
   - Updates weights using calculated gradients
   - Moves model toward better predictions

**Remember**: Zero gradients → Calculate gradients → Apply gradients

# Full Implementaion

We define train_loop that loops over our optimization code, and test_loop that evaluates the model’s performance against our test data.

## Step 9: Define Training and Testing Functions

**Purpose**: Encapsulate the training and evaluation logic.

### `train_loop()` Function:
**What it does**: Trains the model for one epoch
- Loops through all training batches
- For each batch:
  1. Forward pass (get predictions)
  2. Calculate loss
  3. Backward pass (compute gradients)
  4. Update weights
  5. Zero gradients for next iteration
- Prints loss every 100 batches to monitor progress

**Key setting**: `model.train()` - Enables training mode (activates dropout, batch norm)

---

### `test_loop()` Function:
**What it does**: Evaluates model performance on test data
- Loops through all test batches
- Calculates predictions without updating weights
- Computes accuracy and average loss
- Reports overall performance

**Key settings**:
- `model.eval()` - Evaluation mode (disables dropout)
- `torch.no_grad()` - Disables gradient computation (saves memory/time)

**Why separate test loop?**
- Prevents overfitting (model hasn't seen test data)
- Validates that model generalizes to new data

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

We initialize the loss function and optimizer, and pass it to train_loop and test_loop. Feel free to increase the number of epochs to track the model’s improving performance.

## Step 10: Train the Model

**Purpose**: Execute the complete training process.

**Training Process** (10 epochs):
- **Epoch** = One complete pass through training data
- For each epoch:
  1. Train on entire training set (`train_loop`)
  2. Evaluate on entire test set (`test_loop`)
  3. Print progress

**What to Watch**:
- **Training Loss**: Should decrease over epochs
- **Test Accuracy**: Should increase over epochs
- **Gap between train and test**: If test accuracy plateaus while training improves, model may be overfitting

**Expected Behavior**:
- Initial accuracy ~10% (random guessing among 10 classes)
- Final accuracy ~85-88% after 10 epochs

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.306856  [   64/60000]
loss: 2.298582  [ 6464/60000]
loss: 2.283126  [12864/60000]
loss: 2.264119  [19264/60000]
loss: 2.248101  [25664/60000]
loss: 2.226551  [32064/60000]
loss: 2.225410  [38464/60000]
loss: 2.193868  [44864/60000]
loss: 2.198018  [51264/60000]
loss: 2.159197  [57664/60000]
Test Error: 
 Accuracy: 43.6%, Avg loss: 2.157352 

Epoch 2
-------------------------------
loss: 2.168603  [   64/60000]
loss: 2.159842  [ 6464/60000]
loss: 2.107098  [12864/60000]
loss: 2.107127  [19264/60000]
loss: 2.065855  [25664/60000]
loss: 2.007953  [32064/60000]
loss: 2.022113  [38464/60000]
loss: 1.944845  [44864/60000]
loss: 1.959247  [51264/60000]
loss: 1.875185  [57664/60000]
Test Error: 
 Accuracy: 58.2%, Avg loss: 1.881718 

Epoch 3
-------------------------------
loss: 1.915567  [   64/60000]
loss: 1.884475  [ 6464/60000]
loss: 1.778140  [12864/60000]
loss: 1.798797  [19264/60000]
loss: 1.709102  [25664/60000]
loss: 1.660662  [32064/600

# Save and Load the Model

## Step 11: Model Persistence

**Purpose**: Save trained model for later use without retraining.

### Saving the Model:
`torch.save(model, 'model.pth')`
- Saves entire model (architecture + weights)
- `.pth` or `.pt` are common PyTorch model file extensions

**Alternative**: `torch.save(model.state_dict(), 'model.pth')`
- Saves only weights (smaller file, more flexible)
- Requires model architecture definition when loading

---

### Loading the Model:
`torch.load('model.pth', weights_only=False)`
- Loads entire saved model
- `weights_only=False` - Loads full model (not just weights)
- Model is ready to use for predictions immediately

**Why Save Models?**
- Avoid retraining (saves time and compute)
- Deploy models to production
- Share models with others
- Resume training later

In [ ]:
torch.save(model, 'model.pth') # or model.state_dict()?

In [ ]:
model = torch.load('model.pth', weights_only=False)


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

---

## 🤔 Which is Better: `torch.save(model)` vs `torch.save(model.state_dict())`?

### ✅ **RECOMMENDED: `model.state_dict()`**

**Advantages:**
- ✅ **More flexible** - Can load weights into different model versions
- ✅ **Smaller file size** - Only saves weights/parameters, not Python code
- ✅ **More portable** - Works across PyTorch versions better
- ✅ **Safer** - Avoids pickle security issues with untrusted files
- ✅ **Best practice** - Officially recommended by PyTorch documentation
- ✅ **Debugging friendly** - Can inspect weights separately

**Usage Example:**
```python
# Saving
torch.save(model.state_dict(), 'model_weights.pth')

# Loading (requires model architecture definition)
model = NeuralNetwork()  # Create model first
model.load_state_dict(torch.load('model_weights.pth', weights_only=True))
model.eval()  # Set to evaluation mode
```

---

### 🔧 **Alternative: `torch.save(model)`**

**When to use:**
- Quick prototyping and experimentation
- You need to save the entire architecture with weights
- You're sure the model class won't change

**Advantages:**
- ✅ **Simpler loading** - No need to recreate architecture
- ✅ **One-liner** - Quick for testing

**Disadvantages:**
- ❌ **Larger files** - Saves entire model structure
- ❌ **Less portable** - Can break if model class definition changes
- ❌ **Pickle dependency** - Security concerns with untrusted files
- ❌ **Version compatibility issues** - May not work across PyTorch versions

**Usage Example:**
```python
# Saving
torch.save(model, 'model_full.pth')

# Loading (simpler, but less flexible)
model = torch.load('model_full.pth', weights_only=False)
model.eval()
```

---

### 📊 Quick Comparison Table:

| Feature | `state_dict()` | Full Model |
|---------|---------------|------------|
| **File Size** | Smaller ✅ | Larger ❌ |
| **Flexibility** | High ✅ | Low ❌ |
| **Portability** | Better ✅ | Worse ❌ |
| **Loading** | Requires architecture | Direct load ✅ |
| **Best Practice** | Yes ✅ | No ❌ |
| **Security** | Safer ✅ | Pickle risks ❌ |

---

### 💡 **Recommendation for Production:**

Always use `model.state_dict()` for:
- Production deployments
- Sharing models with others
- Long-term storage
- Model versioning

Use full model save only for quick local experiments where convenience matters more than best practices.

---

## 🎓 Summary: Complete PyTorch Workflow

### The 11-Step Process:

1. ✅ **Import** libraries
2. ✅ **Load** datasets
3. ✅ **Create** DataLoaders
4. ✅ **Define** model architecture
5. ✅ **Instantiate** model
6. ✅ **Set** hyperparameters
7. ✅ **Initialize** loss function
8. ✅ **Initialize** optimizer
9. ✅ **Define** train/test loops
10. ✅ **Train** the model
11. ✅ **Save/Load** model

### Key Concepts to Remember:

- **Epochs**: Full passes through the dataset
- **Batches**: Subsets of data processed together
- **Loss**: Measure of prediction error (lower is better)
- **Optimizer**: Updates weights to minimize loss
- **Forward Pass**: Data → Model → Predictions
- **Backward Pass**: Loss → Gradients → Weight Updates
- **Train/Eval Modes**: Different behaviors for training vs testing

### This workflow applies to most deep learning problems!
Just swap out the dataset, model architecture, and hyperparameters for your specific task.

---
**Next Steps**: Experiment with different architectures, optimizers, or datasets to deepen understanding!

---

# 🔍 Making Predictions with the Trained Model

Now that we have a trained model, let's see how to actually use it for making predictions on individual samples. This section demonstrates the complete inference pipeline.

## What We'll Do:
1. Take sample images from the test dataset
2. Feed them to the trained model
3. Examine raw model outputs (logits)
4. Convert logits to predicted class labels
5. Compare predictions with actual labels

This is essential for understanding how your model performs on real data!

## Step 12: Define Class Labels

**Purpose**: Map numeric class indices to human-readable labels.

FashionMNIST has 10 classes (0-9). We need a mapping to understand what the model is predicting.

In [ ]:
# FashionMNIST class labels
class_names = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot"
]

print("FashionMNIST Classes:")
for idx, name in enumerate(class_names):
    print(f"  {idx}: {name}")

## Step 13: Get Sample Data for Testing

**Purpose**: Extract individual samples from the test dataset to make predictions.

We'll get a few test images with their true labels to see how the model performs.

In [ ]:
# Get 5 random samples from test dataset
import random

# Set seed for reproducibility
random.seed(42)
sample_indices = random.sample(range(len(test_data)), 5)

# Extract samples
sample_images = []
sample_labels = []

for idx in sample_indices:
    image, label = test_data[idx]
    sample_images.append(image)
    sample_labels.append(label)
    print(f"Sample {len(sample_images)}: Index {idx}, True Label: {label} ({class_names[label]})")

print(f"\nExtracted {len(sample_images)} samples for testing")

## Step 14: Make Predictions (Single Sample)

**Purpose**: Understand the prediction process step-by-step for ONE sample.

### The Prediction Pipeline:
1. **Prepare Input**: Get image tensor
2. **Add Batch Dimension**: Model expects batches, even for single image
3. **Set Eval Mode**: Disable dropout/batch norm training behavior
4. **Forward Pass**: Feed image through model
5. **Get Logits**: Raw output scores (unnormalized)
6. **Get Probabilities**: Apply softmax to convert logits to probabilities
7. **Get Prediction**: Find class with highest probability

In [ ]:
# Take the first sample
test_image = sample_images[0]
true_label = sample_labels[0]

print(f"True Label: {true_label} ({class_names[true_label]})")
print(f"Image Shape: {test_image.shape}")  # Should be [1, 28, 28]

# Step 1: Set model to evaluation mode
model.eval()

# Step 2: Add batch dimension (model expects [batch_size, channels, height, width])
image_batch = test_image.unsqueeze(0)  # Shape: [1, 1, 28, 28]
print(f"Batch Shape: {image_batch.shape}")

# Step 3: Make prediction (no gradient computation needed)
with torch.no_grad():
    # Forward pass - get raw outputs (logits)
    logits = model(image_batch)

print(f"\n--- Raw Model Output (Logits) ---")
print(f"Logits Shape: {logits.shape}")  # Should be [1, 10]
print(f"Logits: {logits}")
print(f"\nLogits are unnormalized scores for each class.")
print(f"Higher logit = model is more confident about that class.")

## Step 15: Convert Logits to Probabilities

**Purpose**: Transform raw logits into interpretable probabilities using Softmax.

**What is Softmax?**
- Converts logits into probabilities that sum to 1.0
- Higher logits → higher probabilities
- Makes outputs easier to interpret (e.g., "85% confident it's a T-shirt")

In [ ]:
# Apply softmax to convert logits to probabilities
probabilities = torch.nn.functional.softmax(logits, dim=1)

print("--- Probabilities (after Softmax) ---")
print(f"Probabilities Shape: {probabilities.shape}")
print(f"Probabilities: {probabilities}")
print(f"\nSum of probabilities: {probabilities.sum().item():.4f} (should be 1.0)")

# Display probabilities for each class
print("\n--- Probability for Each Class ---")
for idx, prob in enumerate(probabilities[0]):
    print(f"{idx}: {class_names[idx]:15} -> {prob.item()*100:6.2f}%")

## Step 16: Get Final Prediction

**Purpose**: Extract the predicted class (the one with highest probability).

We use `argmax()` to find the index of the maximum value, which corresponds to the predicted class.

In [ ]:
# Get the predicted class (index with highest probability)
predicted_class = logits.argmax(1).item()
confidence = probabilities[0][predicted_class].item()

print("=" * 50)
print("🎯 PREDICTION RESULT")
print("=" * 50)
print(f"True Label:      {true_label} ({class_names[true_label]})")
print(f"Predicted Label: {predicted_class} ({class_names[predicted_class]})")
print(f"Confidence:      {confidence*100:.2f}%")
print(f"Correct:         {'✅ YES' if predicted_class == true_label else '❌ NO'}")
print("=" * 50)

## Step 17: Batch Predictions (Multiple Samples)

**Purpose**: Make predictions for multiple samples at once (more efficient).

Instead of processing images one by one, we can stack them into a batch for faster inference.

In [ ]:
# Stack all sample images into a batch
batch_images = torch.stack(sample_images)  # Shape: [5, 1, 28, 28]
print(f"Batch Shape: {batch_images.shape}")

# Make batch predictions
model.eval()
with torch.no_grad():
    batch_logits = model(batch_images)
    batch_probabilities = torch.nn.functional.softmax(batch_logits, dim=1)
    batch_predictions = batch_logits.argmax(1)

print("\n" + "=" * 70)
print("📊 BATCH PREDICTION RESULTS")
print("=" * 70)

# Display results for each sample
for i in range(len(sample_images)):
    true_label = sample_labels[i]
    pred_label = batch_predictions[i].item()
    confidence = batch_probabilities[i][pred_label].item()

    is_correct = "✅" if pred_label == true_label else "❌"

    print(f"\nSample {i+1}:")
    print(f"  True:       {true_label} ({class_names[true_label]})")
    print(f"  Predicted:  {pred_label} ({class_names[pred_label]})")
    print(f"  Confidence: {confidence*100:.2f}%")
    print(f"  Result:     {is_correct}")

# Calculate accuracy for this batch
correct = (batch_predictions == torch.tensor(sample_labels)).sum().item()
accuracy = correct / len(sample_labels) * 100
print("\n" + "=" * 70)
print(f"Batch Accuracy: {correct}/{len(sample_labels)} = {accuracy:.1f}%")
print("=" * 70)

## 📚 Understanding the Output

### Raw Logits vs Probabilities:

**Logits** (Raw Output):
- Unnormalized scores from the final layer
- Can be any real number (negative or positive)
- Larger value = more confident
- Example: `[2.3, -1.5, 0.8, ...]`

**Probabilities** (After Softmax):
- Normalized values between 0 and 1
- Sum to exactly 1.0
- Interpretable as confidence percentages
- Example: `[0.85, 0.02, 0.05, ...]` → 85%, 2%, 5%, etc.

### Why Use argmax()?
- `argmax()` finds the **index** of the maximum value
- For logits `[2.3, -1.5, 5.1, 0.8]`, `argmax()` returns `2` (the 3rd position)
- This index corresponds to the predicted class label

### Key Takeaways:
1. **Always use `model.eval()`** before making predictions
2. **Use `torch.no_grad()`** to disable gradient computation (saves memory)
3. **Add batch dimension** even for single images: `image.unsqueeze(0)`
4. **Logits → Softmax → Probabilities → argmax → Predicted Class**
5. **Batch processing is faster** than processing one image at a time

---

### Quick Reference - Inference Pipeline:
```python
model.eval()
with torch.no_grad():
    logits = model(image_batch)                    # Raw scores
    probs = torch.nn.functional.softmax(logits, dim=1)  # Probabilities
    prediction = logits.argmax(1)                  # Predicted class
```

```
┌─────────────────────────────────────────────┐
│         1. MODEL (Parameters)               │
│   Weights & Biases (~669K numbers)          │
│   Status: requires_grad=True                │
└─────────────────────────────────────────────┘
                    ↓
         Forward Pass (make predictions)
                    ↓
┌─────────────────────────────────────────────┐
│         2. AUTOGRAD (Gradients)             │
│   Tracks operations                         │
│   Computes: ∂loss/∂parameter                │
│   Called via: loss.backward()               │
└─────────────────────────────────────────────┘
                    ↓
         Gradients stored in .grad
                    ↓
┌─────────────────────────────────────────────┐
│         3. OPTIMIZER (Updates)              │
│   Uses gradients to update parameters       │
│   New_weight = Old_weight - lr * gradient   │
│   Called via: optimizer.step()              │
└─────────────────────────────────────────────┘
```